In [ ]:
from PIL import Image, ImageDraw, ImageFont
from noaa_sdk import noaa
import astral
import datetime
import os

In [ ]:
#Open the NWS API in python to get the current weather condition and RH
n = noaa.NOAA()
observations = n.get_observations('02128','US')

In [ ]:
obs = next(observations)

In [ ]:
temp = obs['temperature'] #degC
temp = (temp['value'] * 9/5)+32
dp = obs['dewpoint'] #degC
dp = (dp['value'] * 9/5) + 32
description = obs['textDescription']
wdir = obs['windDirection'] #degrees
wspd = obs['windSpeed'] #m/s
if(wspd['value'] == None):
    wspd['value'] = 0.00
wspd = wspd['value'] * 2.23694
wgust = obs['windGust'] # m/s
press = obs['seaLevelPressure'] #in Pa
time = obs['timestamp'] #in Datetime
vis = obs['visibility'] #Note in meters
if(vis['value'] == None):
    vis['value'] = 0.00
rh = obs['relativeHumidity'] #%
precip_last_hour = obs['precipitationLastHour'] #meters
if(precip_last_hour['value'] == None):
    precip_last_hour = 0.00
else:
    precip_last_hour = precip_last_hour['value'] * 39.37

Choose the image based on the description

In [ ]:
if("Cloudy" in description):
    image = "cloudy.png"
elif("Fair" in description):
    image = "clear.png"
elif("Clear" in description):
    image = "clear.png"
elif("Overcast" in description):
    image = "overcast.png"
elif("Fog" in description):
    image = "fog.png"
elif("Freezing" in description):
    image = "freezing.png"
elif("Rain" in description):
    image = "rain.png"
elif("Ice" in description):
    image = "ice.png"
elif("Snow" in description):
    image = "snow.png"
elif("Breezy" in description):
    image = "windy.png"
elif("Showers" in description):
    image = "rain_showers.png"
else:
    image = "clear.png"

In [ ]:
filepath = "/home/metweb/public_html/UMLStorm/nws_kbox/"
wind_dir = wdir['value']

In [ ]:
if(wind_dir >= 349 or wind_dir <= 11):
    filepath = filepath + 'North.png'
elif(wind_dir >= 12 and wind_dir <= 78):
    filepath = filepath + 'Northeast.png'
elif(wind_dir >= 79 and wind_dir <= 101):
    filepath = filepath + 'East.png'
elif(wind_dir >= 102 and wind_dir <=168):
    filepath = filepath + 'Southeast.png'
elif(wind_dir >= 169 and wind_dir <= 191):
    filepath = filepath + 'South.png'
elif(wind_dir >= 192 and wind_dir <= 258):
    filepath = filepath + 'Southwest.png'
elif(wind_dir >= 259 and wind_dir <= 281):
    filepath = filepath + 'West.png'
elif(wind_dir >= 281 and wind_dir <= 348):
    filepath = filepath + 'Northwest.png'
else:
    filepath = ''

In [ ]:
FT_PER_METRE = 3.2808399
# Construct our location.  Longitude west and latitude south are
# negative.
los_altos = astral.Location(info=("Boston", "USA", 42.36,
                                  -71.01, "America/New_York",
                                  20/FT_PER_METRE))

In [ ]:
# "civil", which means 6 degrees below the horizon, is the default
# value for computing dawn and dusk.  But this usage shows how to
# set it before calculation.
los_altos.solar_depression = "civil"

In [ ]:
tomorrow = datetime.date.today()

In [ ]:
result = los_altos.sun(date=tomorrow)

In [ ]:
sunrise = result['sunrise'].strftime('%H:%M %p')
sunset = result['sunset'].strftime('%H:%M')
sunset = result['sunset'].strptime(sunset,'%H:%M')
sunset = sunset.strftime('%I:%M %p')

In [ ]:
wind_image = Image.open(filepath)
wind_image = wind_image.resize([65,65])

In [ ]:
iconimage = Image.open(image)
image2 = image
if(image == 'rain.png' or image == 'snow.png' or image =='ice.png' or image == 'freezing.png' or image == 'fog.png' or image == 'windy.png' or image == 'overcast.png' or image == 'cloudy.png'):
    iconimage = iconimage.resize([300,315])
else:
    iconimage = iconimage.resize([350,350])

In [ ]:
filename = "/home/metweb/public_html/UMLStorm/nws_kbox/logan_wx.png"
image = Image.open(filename)
draw = ImageDraw.Draw(image)
#font = ImageFont.truetype(<font-file>, <font-size>)
font = ImageFont.truetype("LexendDeca-Regular.ttf", 48)
font2 = ImageFont.truetype("LexendDeca-Regular.ttf", 88)
font3 = ImageFont.truetype("LexendDeca-Regular.ttf", 55)
font4 = ImageFont.truetype("LexendDeca-Regular.ttf", 55)
font5 = ImageFont.truetype("LexendDeca-Regular.ttf", 32)
font6 = ImageFont.truetype("LexendDeca-Regular.ttf", 40)
#draw.text((x, y),"Sample Text",(r,g,b))
draw.text((140,90),"Boston Current Conditions", (255,255,255),font = font)
if(len(str(round(wspd))) == 2):
    draw.text((225,200),str(round(wspd)),(255,255,255),font=font3)
else:
    draw.text((245, 200), str(round(wspd)), (255, 255, 255), font=font3)
draw.text((605,213),str(round(rh['value'])),(255,255,255),font=font3)
draw.text((180,411),str(round(temp)) + ' F',(255,255,255),font=font4)
draw.text((610,411),str(round(dp)) + " F",(255,255,255),font=font4)
draw.text((333,436),"Precip: " + str(round(precip_last_hour,3)) + '"',(255,255,255),font=font6)
draw.text((370,516),sunrise,(255,255,255),font=font5)
draw.text((580,516),sunset,(255,255,255),font=font5)
draw.text((170,516),str(round(vis['value']/1609,2)) + ' miles',(255,255,255),font=font5)
image.paste(wind_image, [165,215],wind_image.convert('RGBA'))
if(image2 == 'rain.png' or image2 == 'snow.png' or image2 =='ice.png' or image2 == 'freezing.png' or image2 == 'fog.png' or image2 == 'windy.png' or image2 == 'overcast.png' or image2 == 'cloudy.png'):
    image.paste(iconimage, [290,140], iconimage.convert('RGBA'))
else:
    image.paste(iconimage, [255,140], iconimage.convert('RGBA'))
image.save('current.png')